In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import copy

# Setup options
options = Options()
# options.add_argument("--headless")  # Uncomment after testing
options.add_argument("--window-size=1920,1080")

# Start the browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)

driver.get("http://www.sprengnetter.de/gutachter-sachverstaendigenverzeichnis/")
time.sleep(2)

response=[]

In [19]:
# Accept cookie banner
try:
    cookie = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button[@id='CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll']"))
    )
    driver.execute_script("arguments[0].click();", cookie)
    time.sleep(1)
except:
    print("✅ Cookie banner not found or already accepted.")

In [ ]:
def extract_results():

    try:
        rows = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//tbody[@class='row-striping row-hover']/tr"))
        )

        for row in rows:
            try:
                # Get all columns in the row
                columns = row.find_elements(By.TAG_NAME, "td")

                # Skip rows that are empty or invalid
                if len(columns) < 6:
                    continue

                cert = columns[0].text.strip()
                name = columns[1].text.strip()
                phone = columns[2].text.strip()
                street = columns[3].text.strip()
                postcode = columns[4].text.strip()
                location = columns[5].text.strip()

                full_address = f"{street}, {postcode} {location}"

                response.append({
                    "Full Name": name,
                    "Address": full_address,
                    "Phone Number": phone,
                    "Email Address": "",     # Not available
                    "Website": "",           # Not available
                    "Certification Type / Details": cert,
                    "Source Directory Name": "SPRENGNETTER"
                })

            except Exception as row_err:
                print(f"❗ Error extracting a row: {str(row_err)}")

    except TimeoutException:
        print("⏱️ Timeout: No rows found; check table structure or wait time.")
    except Exception as e:
        print(f"❗ Unexpected error extracting results: {str(e)}")



In [ ]:
while True:
    extract_results()
    try:
        next_button = driver.find_element(By.XPATH, '//button[@class="dt-paging-button next"]')
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(3)
    except Exception as e:
        print(f"🔁 All the Data Extract from the website")
        break

In [22]:
print(response)



[{'Full Name': 'Boris Coenen', 'Address': 'Eduard-Bilz-Straße 55a, 01445 Radebeul', 'Phone Number': '0351/82618697', 'Email Address': '', 'Website': '', 'Certification Type / Details': 'ZIS Sprengnetter Zert (S)', 'Source Directory Name': 'DIA_ZERT'}, {'Full Name': 'Margit Paul', 'Address': 'Königsteiner Straße 22, 01796 Pirna', 'Phone Number': '03501/585071', 'Email Address': '', 'Website': '', 'Certification Type / Details': 'ZIS Sprengnetter Zert (WG)', 'Source Directory Name': 'DIA_ZERT'}, {'Full Name': 'Knut Hiller-Schleehuber', 'Address': 'Dr.-Peter Jordan-Straße 23, 02625 Bautzen', 'Phone Number': '0359/1607922', 'Email Address': '', 'Website': '', 'Certification Type / Details': 'ZIS Sprengnetter Zert (AI)', 'Source Directory Name': 'DIA_ZERT'}, {'Full Name': 'Lutz Schneider', 'Address': 'Zittauer Straße 12, 02681 Wilthen', 'Phone Number': '0359/231908', 'Email Address': '', 'Website': '', 'Certification Type / Details': 'ZIS Sprengnetter Zert (S)', 'Source Directory Name': 'DI

In [23]:
backup_response = copy.deepcopy(response)
print(f"🔎 Original: {len(backup_response)} entries")

🔎 Original: 314 entries


In [24]:
# Clean: remove entries without an address
response = [entry for entry in response if entry.get("Address")]
print(f"✅ After cleaning: {len(response)} entries")

✅ After cleaning: 314 entries


In [ ]:
df = pd.DataFrame(response)
df.drop_duplicates(inplace=True)
df.to_csv("SPRENGNETTER.ipynb_results.csv", index=False)
df.to_excel("SPRENGNETTER.ipynb_results.xlsx", index=False)